In [68]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [154]:
def getRealId(id):
	if 'wikipedia.org' in id:
		id = "/wiki/{}".format(id.split('/')[-1])
	response = requests.get(
		url="https://en.wikipedia.org{}".format(id),
	)
	soup = BeautifulSoup(response.content, 'html.parser')
	link = soup.find('link', rel='canonical').get('href').split('/')[-1]
	return "/wiki/{}".format(link)


print(getRealId("https://fr.wikipedia.org/wiki/Aubry_II_de_M%C3%A2con"))

/wiki/Aubry_II_de_M%C3%A2con


In [160]:
response = requests.get(
	url="https://en.wikipedia.org/wiki/Nomen_nescio",
)
soup = BeautifulSoup(response.content, 'html.parser')

infobox = soup.find("table", class_="infobox")
print(infobox)

None


In [161]:
person['name'] = soup.find("span", class_="mw-page-title-main").text

AttributeError: 'NoneType' object has no attribute 'text'

In [162]:
def extractDateFromStr(dateDateStr, person, birth=True):
	# Check for the presence of "c." prefix
	if "c." in dateDateStr:
		dateDateStr = dateDateStr.replace("c.", "").strip()
	if '(' in dateDateStr:
		dateDateStr = dateDateStr.split('(')[0]

	# Extract day, month, and year
	date_parts = dateDateStr.split()
	day = None
	month = None
	year = None

	print(date_parts)
	# Process the date parts based on their availability
	if len(date_parts) == 3:
		day, month, year = date_parts
	elif len(date_parts) == 2:
		# Handle cases like "April 747" where the day is missing
		if date_parts[1].isdigit():
			month, year = date_parts
		else:
			month, year = date_parts[::-1]  # Reverse the order
	elif len(date_parts) == 1:
		# Handle cases like "747" where only the year is available
		year = date_parts[0]

	print(month, year)
	# Convert year to an integer if available
	if year:
		year = int(year)

	# Create a datetime object with the available date parts
	if year:
		person['{}Date'.format("birth" if birth else "death")] = datetime(year=year, month=datetime.strptime(month, "%B").month,
							 day=int(day)) if day and month else datetime(year=year, month=1, day=1)

def extractDate(infobox, person, birth=True):
	dateRow = infobox.find("th", text="Born").parent if infobox.find("th", text="Born") else None
	if not birth:
		dateRow = infobox.find("th", text="Died").parent if infobox.find("th", text="Died") else None
	if dateRow:
		print(dateRow)
		dateDateStrList = dateRow.find("td").contents
		print(dateDateStrList)
		for dateDateStr in dateDateStrList:
			try:
				extractDateFromStr(dateDateStr, person, birth=birth)
			except:
				print("Bad Date String: {}".format(dateDateStr))
		person['{}Place'.format("birth" if birth else "death")] = [a for a in dateRow.find_all("a") if a.get("title")][0].get("href") if len([a for a in dateRow.find_all("a") if a.get("title")]) > 0 else None
person = {}
extractDate(infobox, person)
extractDate(infobox, person, birth=False)
print(person)

AttributeError: 'NoneType' object has no attribute 'find'

In [115]:
extractDateFromStr("799 (aged 74)", person)

['799']
None 799


In [27]:
deathRow = infobox.find("th", text="Died").parent
print(deathRow)
deathDate = deathRow.find("td")
print(deathDate.contents[0])
deathPlace = [a for a in deathRow.find_all("a") if a.get("title")][0].get("href")
print(deathPlace)

<tr><th class="infobox-label" scope="row">Died</th><td class="infobox-data">28 January 814 <span class="nowrap">(aged 66)</span><sup class="reference" id="cite_ref-Charlemagne_Britannica_2-1"><a href="#cite_note-Charlemagne_Britannica-2">[2]</a></sup><br/>Aachen, <a href="/wiki/Francia" title="Francia">Francia</a></td></tr>
28 January 814 
/wiki/Francia


In [86]:
if infobox.find("th", text="Spouses"):
	spouseRow = infobox.find("th", text="Spouses").parent
	print(spouseRow)
	spouseList = [spouse.find("a").get("href") for spouse in spouseRow.findAll("li") if spouse.find("a") and spouse.find("a").get("href")]
	print(spouseList)
else:
	spouseRow = infobox.find("th", text="Spouse").parent
	print(spouseRow)
	spouseList = [spouseRow.find('a').get('href')]
	print(spouseList)

<tr><th class="infobox-label" scope="row">Spouse</th><td class="infobox-data"><a href="/wiki/Charlemagne" title="Charlemagne">Charlemagne</a> (m. <abbr title="circa">c.</abbr><span style="white-space:nowrap;"> 783</span>)</td></tr>
['/wiki/Charlemagne']


In [36]:
issueRow = infobox.find(lambda tag: tag.name == 'th' and 'Issue' in tag.text).parent
print(issueRow)
issueList = [issue.find("a").get("href") for issue in issueRow.findAll("li") if issue.find("a") and issue.find("a").get("href")]
print(issueList)

<tr><th class="infobox-label" scope="row">Issue<br/><span style="font-weight:normal"><i><a href="#Wives,_concubines,_and_children">Among others</a></i></span></th><td class="infobox-data"><link href="mw-data:TemplateStyles:r1126788409" rel="mw-deduplicated-inline-style"/><div class="plainlist">
<ul><li><a href="/wiki/Pepin_the_Hunchback" title="Pepin the Hunchback">Pepin the Hunchback</a></li>
<li><a href="/wiki/Charles_the_Younger" title="Charles the Younger">Charles the Younger</a></li>
<li><a href="/wiki/Pepin_of_Italy" title="Pepin of Italy">Pepin of Italy</a></li>
<li><a href="/wiki/Louis_the_Pious" title="Louis the Pious">Louis the Pious</a></li></ul>
</div></td></tr>
['/wiki/Pepin_the_Hunchback', '/wiki/Charles_the_Younger', '/wiki/Pepin_of_Italy', '/wiki/Louis_the_Pious']


In [40]:
dynastyRow = infobox.find("th", text="Dynasty").parent
print(dynastyRow)
dynasty = dynastyRow.find("td").find("a").get("href")
print(dynasty)

<tr><th class="infobox-label" scope="row"><a href="/wiki/Dynasty" title="Dynasty">Dynasty</a></th><td class="infobox-data"><a href="/wiki/Carolingian_dynasty" title="Carolingian dynasty">Carolingian</a></td></tr>
/wiki/Carolingian_dynasty


In [90]:
fatherRow = infobox.find("th", text="Father").parent
print(fatherRow)
father = fatherRow.find("td").find("a").get("href") if fatherRow.find("td").find("a") else fatherRow.find("td").text
print(father)

<tr><th class="infobox-label" scope="row">Father</th><td class="infobox-data">East Frankish Count Rudolph</td></tr>
East Frankish Count Rudolph


In [42]:
motherRow = infobox.find("th", text="Mother").parent
print(motherRow)
mother = motherRow.find("td").find("a").get("href")
print(mother)

<tr><th class="infobox-label" scope="row">Mother</th><td class="infobox-data"><a href="/wiki/Bertrada_of_Laon" title="Bertrada of Laon">Bertrada of Laon</a></td></tr>
/wiki/Bertrada_of_Laon


In [159]:
houseRow = infobox.find("th", text="House").parent
print(houseRow)
house = houseRow.find("td").find("a").get("href") if houseRow.find("td").find("a") else houseRow.find("td").text
print(house)

<tr><th class="infobox-label" scope="row"><a href="/wiki/Dynasty" title="Dynasty">House</a></th><td class="infobox-data"><a class="mw-redirect" href="/wiki/Capet" title="Capet">Capet</a></td></tr>
/wiki/Capet


In [157]:
religionRow = infobox.find("th", text="Religion").parent
print(religionRow)
religion = religionRow.find("td").find("a").get("href") if religionRow.find("td").find("a") else religionRow.find("td").text
print(religion)

<tr><th class="infobox-label" scope="row">Religion</th><td class="infobox-data">Roman Catholic</td></tr>
Roman Catholic


In [45]:
picture = infobox.find("img").get("src")
print(picture)

//upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Charlemagne_denier_Mayence_812_814.jpg/225px-Charlemagne_denier_Mayence_812_814.jpg


In [46]:
name = infobox.find("th").text
print(name)

Charlemagne


In [147]:
def getTitle(infobox, person):
	if infobox.find("td", class_="infobox-subheader"):
		person['title'] = infobox.find("td", class_="infobox-subheader").find('a').get('href') if  infobox.find("td", class_="infobox-subheader").find('a') else  infobox.find("td", class_="infobox-subheader").text
	elif infobox.find("th", attrs={"style": "background-color: #e4dcf6;line-height:normal;padding:0.2em 0.2em"}):
		person['title'] = infobox.find("th", attrs={"style": "background-color: #e4dcf6;line-height:normal;padding:0.2em 0.2em"}).find('a').get('href') if infobox.find("th", attrs={"style": "background-color: #e4dcf6;line-height:normal;padding:0.2em 0.2em"}).find('a') else infobox.find("th", attrs={"style": "background-color: #e4dcf6;line-height:normal;padding:0.2em 0.2em"}).text
person = {}
getTitle(infobox, person)
print(person)

{'title': 'Countess Palatine of Lotharingia'}
